<a href="https://colab.research.google.com/github/Gricay-vasily/project_9_DCGAN/blob/main/Learning_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Навчання Deep Convolutional Generative Adversarial Network (DCGAN) на завантажених даних Cifar

## Завантаження і попередня підготовка Дата-сетів

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

### Зчитування мета-даних та суперкласів

Посилання на дані

In [ ]:
URL_DIR = "https://www.cs.toronto.edu/~kriz/"
# FILE_NAME_CIFAR_10 = "cifar-10-python.tar.gz"
FILE_NAME_CIFAR_100 = "cifar-100-python.tar.gz"

Завантажимо дані

In [ ]:
def load_and_unzip_cifar(url:str, file_name:str):
  '''Завантаження і розпакування файлу-архіву, якщо ще цього не зроблено'''
  # Завантаження даних
  from pathlib import Path
  full_url_to_load = url + file_name
  print(full_url_to_load)
  if Path(file_name).exists() and Path(file_name).is_file():
    print(f"Файл {file_name} вже є. Немає потреби його завантажувати!")
  else:
    print(f"Заватажуємо файл {file_name} ")
    !wget $full_url_to_load
  # Розпакування даних
  # Створення робочої папки через stem i split - відсікання суфіксів
  work_dir = Path(str((Path() / Path(file_name).stem)).split(".")[0])
  # print(work_dir)
  if work_dir.exists() and work_dir.is_dir():
    print(f"Директорія {work_dir} вже існує, перезаписуємо файли")
    # Перезаписуємо, бо є файл *txt, який не дає видалити папку
    !tar -xzvf $file_name
  else:
    print(f"Розпакуємо файл {file_name}")
    !tar -xzvf $file_name

load_and_unzip_cifar(URL_DIR, FILE_NAME_CIFAR_100)

In [ ]:
def unpickle(file):
  '''Десеріалізація даних з pickle-файлу'''
  import pickle
  with open(file, 'rb') as fo:
      dict = pickle.load(fo, encoding='bytes')
  return dict

In [ ]:
metadata_path = './cifar-100-python/meta' # шлях до даних
metadata = unpickle(metadata_path)
superclass_dict = dict(list(enumerate(metadata[b'coarse_label_names'])))

### Завантаження тренувальної та тестуувальної вибірок (використовуючи суперкласи):

In [ ]:
data_pre_path = './cifar-100-python/' # шлях до даних
# шляхи до файлів
data_train_path = data_pre_path + 'train'
data_test_path = data_pre_path + 'test'
# Зчитуємо словники
data_train_dict = unpickle(data_train_path)
data_test_dict = unpickle(data_test_path)
# Отримуємо доні (вибираємо coarse_labels щоб отримати всі 100 класів)
data_train = data_train_dict[b'data']
label_train = np.array(data_train_dict[b'coarse_labels'])
data_test = data_test_dict[b'data']
label_test = np.array(data_test_dict[b'coarse_labels'])

Поверхнево дослідимо дані

In [ ]:
type(data_train.shape), type(data_test.shape)

In [ ]:
data_train.shape, data_test.shape

In [ ]:
np.info(data_train)

In [ ]:
data_train.reshape(len(data_train), 3, 32, 32).shape, data_test.reshape(len(data_test), 3, 32, 32).shape

### Зміна розмірності зображень - виконувати лише раз у колабі!!!

In [ ]:
# Транспонується саме (0,2,3,1) - щоб отримати:
# 0 - Позицію картинки
# 2 - Значення висоти в пікселях
# 3 - Значення ширини в пікселях
# 1 - значення кольорів у RGB
# Якщо набрати (0,3,2,1) - картинка перевертається на 90град - висота і ширина міняються місцями

data_train = data_train.reshape(len(data_train), 3, 32, 32).transpose(0,2,3,1)
data_test = data_test.reshape(len(data_test), 3, 32, 32).transpose(0,2,3,1)

In [ ]:
np.info(data_train)

In [ ]:
_ = random.randint(0,len(data_train))
print(f"For picture #{_} - {label_train[_]}({superclass_dict[label_train[_]]}):")
fig = plt.figure(figsize=(1,1))
fig.add_subplot(1,1,1)
plt.imshow(data_train[_])
plt.show()

Трохи більше картинок з тренувальної та тестувальної вибірок...

In [ ]:
def show_pictures_n_m(data,n,m:int):
    fig = plt.figure(figsize=(max(n,m),max(n,m)))
    for _ in range (n*m):
        fig.add_subplot(n,m,_+1)
        plt.imshow(data[_])
    plt.show()

show_pictures_n_m(data_train,5,5)
show_pictures_n_m(data_test,3,3)

## Подальша підготовка даних для тренувань мережі

In [ ]:
# Також виконується приведення даних до типу float32
# та нормалізація даних до діапазону наприклад [0, 1]
# data_train = data_train.astype("float32")
# data_test = data_test.astype("float32")
# data_train = (data_train - 255)/ 255
# data_test = (data_test - 255)/ 255

# np.info(data_train)

In [ ]:
import tensorflow as tf
tf.__version__